# Post-processing using LMGC90 API

The current recommended way to design any post-processing within LMGC90 is in fact to use the python API inside the computation script to extract the desired value along the way.

As it may not be known beforehand exactly what may be needed, it is often desired to first run the computation, and only then design the post-processing step of interest. This can be done *a posteriori* as long as the LMGC90 database has been saved during the computation (thanks to the `chipy.WriteOut` function) whether with the old fashionned text files or the HDF5 binary files.

Here to not duplicate irrelevant informations, it is assumed that the [previous notebook](internalPostpro.ipynb) has been run at least once and that the *DATBOX* input directory and the *lmgc90.h5* output file associated to the computation are present.

### Basic idea

The idea is to reload a particular time step to be able to get information exactly like during a computation. So the first thing is to read the DATBOX and do all the needed initialization (in most tutorial examples, all these are hidden under the single `computation.initialize` command). Here is the list of needed commands:

In [ ]:
from pylmgc90 import chipy

dim = 2

# reading the DATBOX and loadings
chipy.Initialize()
chipy.checkDirectories()

chipy.Initialize()

chipy.SetDimension(dim)

chipy.ReadBodies()
chipy.ReadBehaviours()

chipy.LoadTactors()
chipy.LoadBehaviours()

chipy.ReadDrivenDof()

Now a particular record can be read from the file to change the state of the database. It is important to differentiate *record* and *step*; if and only if all the time steps have been saved during the computation these indices may look identical. In fact the *step* starts at 0 (which the initial state) and is an increasing integer describing each time step computed. The *record* is also an increasing integer number, but starting at 1 ! And it counts the number of record of the database made.

As such, since the `freq_write` of the previous step was `10` and `dt` had the value $10^{-3}$ what can be obtained is:

In [ ]:
# reading first saved step:
h5_file   = 'lmgc90.h5'
id_record = 1
chipy.ReadIni(id_record,h5_file)

# start to check database content:
nstep = chipy.TimeEvolution_GetStep()
print(nstep)
time  = chipy.TimeEvolution_GetTime()
print(time)

### Bodies access

Now the accessor of the database can be used. The main one is [`chipy.RBDY2_GetBodyVector`](https://lmgc90.pages-git-xen.lmgc.univ-montp2.fr/lmgc90_dev/chipy_lmgc90.html#pylmgc90.chipy.lmgc90.RBDY2_GetBodyVector) for 2D rigids.

This way of interacting with the database is a little more technical, since it assumes that the user knows a little how data are organized in the core of the sofwtare and how to extract the relevant data. As such the aforementionned accessor allow to access the rigid bodies. To access a particular geometry, one must knows how to get back from a contactor number to the id of its supporting body.

For bodies, the only information which is stored within the files are the coordinates and velocity. So to get back the external forces and reaction on the body, they must be recomputed from the gravity field and the contact network. Hence the [`chipy.RBDY2_ComputeMass`](/https://lmgc90.pages-git-xen.lmgc.univ-montp2.fr/lmgc90_dev/chipy_lmgc90.html#pylmgc90.chipy.lmgc90.RBDY2_ComputeMass),
[`chipy.RBDY2_ComputeFext`](https://lmgc90.pages-git-xen.lmgc.univ-montp2.fr/lmgc90_dev/chipy_lmgc90.html#pylmgc90.chipy.lmgc90.RBDY2_ComputeFext)
and [`chipy.inter_handler_2D_computeRnod`](https://lmgc90.pages-git-xen.lmgc.univ-montp2.fr/lmgc90_dev/chipy_lmgc90.html#pylmgc90.chipy.lmgc90.inter_handler_2D_computeRnod) calls (the last one having effect only when reading HDF5 files). A simple example would be to look for the body with maximum velocity:

In [ ]:
import numpy as np


v = chipy.RBDY2_GetAllBodyVector('Vbeg_')

norm_v = np.linalg.norm(v, axis=1)
max_idx = np.argmax( norm_v )

print( f"Max velocity {v[max_idx]} of body {max_idx+1}")

In [ ]:
i_rbdy2 = 12
fext = chipy.RBDY2_GetBodyVector('Fext_', i_rbdy2)
reac = chipy.RBDY2_GetBodyVector('Reac_', i_rbdy2)
print( f"Fext of body {i_rbdy2}: {fext}" )
print( f"Reac of body {i_rbdy2}: {reac}" )

chipy.RBDY2_ComputeMass()
chipy.RBDY2_ComputeFext()
chipy.inter_handler_2D_computeRnod()
fext = chipy.RBDY2_GetBodyVector('Fext_', i_rbdy2)
reac = chipy.RBDY2_GetBodyVector('Reac_', i_rbdy2)
print( f"Fext of body {i_rbdy2}: {fext}" )
print( f"Reac of body {i_rbdy2}: {reac}" )

### Interactions access

For the interactions, there a single getter: [`chipy.getInteractions`](https://lmgc90.pages-git-xen.lmgc.univ-montp2.fr/lmgc90_dev/chipy_macro.html#pylmgc90.chipy.macro.getInteractions)

This array is a [structured numpy array](https://numpy.org/doc/stable/user/basics.rec.html) and allow
to access all relevant data of the interactions in a single numpy array.

A simple example would be to look for the maximum reaction value and identify which bodies are involved:

In [ ]:
inters = chipy.getInteractions()
#print(inters.shape)

# computing the norm of the reaction for each
norm_reac = np.linalg.norm( inters['rl'], axis=1)

# find the max:
i_max = np.argmax( norm_reac )
print( f"maximum norm reaction interaction: {i_max+1} of value {norm_reac[i_max]}")

print( f"between rigid bodies {inters[i_max][['icdbdy','ianbdy']]}")

TODO : explanation difference between this and verlet ?

### A posteriori visualisation

Until now, a single step has been reloaded, but there is nothing preventing from reload different time steps. And, in case the simulation is heavy (in term of RAM and computing power), it is advised to not generate the paraview visualisation files.

But in case of need, these visualisation files can be generated afterwards, again provided that there is either the *OUTBOX* text files or the HDF5 binary output file:

In [ ]:
h5_file   = 'lmgc90.h5'
nb_record  = 30

chipy.OpenDisplayFiles()
chipy.ComputeMass()

for k in range(1,nb_record+1):

    chipy.ReadIni(k,h5_file)
    chipy.ComputeFext()
    chipy.ComputeRnod()
    chipy.WriteDisplayFiles(1)

chipy.CloseDisplayFiles()

In [ ]:
!paraview